We are going to do the following

Check the scores for keywords

In [12]:
import pandas as pd
from IPython.display import display
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk import word_tokenize

import re
stop = stopwords.words('english')


#keywords
keywords = ['false','misleading','allegation','hoax','rumour','rumor','denied','inaccurate','dispute','claimed']
mis_dis_kw = list(map(porter_stemmer.stem,keywords))

si_kw = ['retaliate','fear','hate','tension','burn','forc','threat','protest','terrorist']
si_kw = list(map(porter_stemmer.stem,si_kw))

#conspiracy_theory
ct_keywords = ['plot','denied','inaccurate','dispute','claimed']
ct_kw = list(map(porter_stemmer.stem,ct_keywords))

#rumor
r_keywords = ['rumour','rumor','rumours']
r_kw = list(map(porter_stemmer.stem,ct_keywords))

def keyword_score(p,kw):
    kwa_score = 0
    for i in kw:
        kwa_score += len(re.findall(i,p))
    return kwa_score

split to 3 sets change to lower case, remove punctuation and remove stopwords

In [13]:
import glob

path = r'C:\Users\User\Desktop\Terrorism and Fake News Project\00_Pipeline_0\2.Datasets\h5 files'

all_files = glob.glob(path + "/*.h5")

li = []

for filename in all_files:
    li.append(filename)

In [14]:
li[0]

'C:\\Users\\User\\Desktop\\Terrorism and Fake News Project\\00_Pipeline_0\\2.Datasets\\h5 files\\201501010014_12574.h5'

In [15]:
pd.read_pickle(li[0])

0    https://www.news.com.au/world/breaking-news/tw...
1                                         201501010014
2    [Skip to:Main ContentSite NavigationSite foote...
Name: 4, dtype: object

In [20]:
from nltk.stem.porter import PorterStemmer

stop = stopwords.words('english')
porter_stemmer = PorterStemmer()
#this is done so that i can apply lambda function without too much for loops by mapping the stem function to the words
def mapstem(lst):
    return [porter_stemmer.stem(i) for i in lst]

#takes a series and processes the text
def txt_preprocess(series):
    #convert to df
    df = series.to_frame().transpose()
    df.columns=['gtd_id','link','txts']
    #melting list
    df = df.txts.apply(pd.Series)\
    .merge(df, right_index = True, left_index = True)\
    .drop(["txts"], axis = 1)\
    .melt(id_vars = ['link','gtd_id'], value_name = "txt") \
    .drop("variable", axis = 1) \
    .drop_duplicates()\
    .dropna()
    #display(df.tail())
    #display(type(df['txt'][0]))
    
    col = 'txt'
    #lower case
    df[col] = df[col].apply(lambda x: " ".join(x.lower() for x in x.split()))
    #remove "\n"
    df = df.replace('\n',' ', regex=True)
    #remove punctuation
    df[col] = df[col].str.replace('[^\w\s]','')
    #remove stopwords
    df[col] = df[col].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    
    #tokenize words
    df[col] = df[col].apply(word_tokenize)
    #stem words
    df[col] = df[col].apply(lambda x: mapstem(x))
    #stitching
    df[col] = df[col].apply(lambda x: ' '.join(x))
    
    return df

#this method quite computationally expensive cos you are running things at o(n^2)
def label_h5(h5):
    #if data exists as a series
    series=pd.read_pickle(h5) #.drop('headline',axis=1)
    frame = txt_preprocess(series)
    
    #if data exists as a dataframe
    
    #label rumour
    df2 = pd.DataFrame()
    for i in range(len(frame)):
        txt= frame['txt'].iloc[i]
        #print('score: '+ str(keyword_score(txt,mis_dis_kw)))
        df2a = pd.DataFrame([keyword_score(txt,r_kw)])
        df2a.columns=['r_score']
        
        df2= pd.concat([df2,df2a], sort=True) #sort=True to silence warnings
    df2 = df2.reset_index(drop=True)
    
    #label conspiracy theory
    df3 = pd.DataFrame()
    for i in range(len(frame)):
        txt= frame['txt'].iloc[i]
        df3a= pd.DataFrame([keyword_score(txt,ct_kw)])
        df3a.columns=['ct_score']
        df3= pd.concat([df3,df3a], sort=True)
    df3 = df3.reset_index(drop=True)

    
    frame=pd.concat([df2,df3, frame], axis=1, sort=True)
    frame = frame.dropna()
    
    return frame

In [22]:
kw=pd.DataFrame()
kw.to_csv('keyword_score_2.csv')

In [21]:
for i in range(10):

    print(i,end='\r')
    d=label_h5(li[i])
    ndf = pd.DataFrame([d['ct_score'].max(),
                        d['r_score'].max(),
                        d['link'][0],d['gtd_id'][0]]).transpose()
    ndf.columns=['mis_dis_score', 'si_score', 'link', 'gtd_id']

    ndf.to_csv('keyword_score.csv',mode='a',header=None)
    '''
    except Exception as e: #this is bad code
        print(e)
        continue
    '''

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
s_df = pd.DataFrame()
for i in len(df):
    text = df[1].iloc[i]
    score = poh_score(poh_list,text)
    s_df = pd.concat(s_df,score)
s_df